In [8]:
import time

In [1]:
original_prica = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [2]:
from collections import defaultdict

In [3]:
price = defaultdict(int)

In [4]:
for i,p in enumerate(original_prica):
    price[i+1] = p

In [5]:
price[11]

35

## Get the max splitting by enumerate

In [6]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

In [7]:
def r(n):
    
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )

In [10]:
%%time
r(10)

Wall time: 16 ms


30

In [11]:
%%time
r(15)

Wall time: 3.5 s


45

In [12]:
import time

In [13]:
#@get_time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [84]:
start = time.time()
print(fibonacci(38))
end = time.time()
print(end-start)

39088169
7.479214906692505


In [15]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [82]:
start = time.time()
print(fibonacci_op(100))
end = time.time()
print(end-start)

489526700523968661124
0.0009975433349609375


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [85]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [86]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [87]:
def f():
    print('HELLO')

In [88]:
f = f1(f)

In [89]:
print(f.__name__)

wrapper


In [90]:
@f1
def g(a):
    print(a)

In [91]:
g('hello')

Started
hello
Ended


In [92]:
def k(*arg,**kwargs):
    print(kwargs)

In [93]:
k(6,b=5)

{'b': 5}


In [97]:
from functools import wraps

In [98]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [99]:
solution = {}

In [100]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [101]:
r(20)

60

In [102]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             20: 0,
             19: 0,
             18: 0,
             17: 0,
             16: 0})

In [103]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

# How do we parse solution?¶

In [104]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [105]:
r(24)

75

In [110]:
parse_solution(24)

[11, 11, 2]

# Edit Distance

In [111]:
solution = {}

In [11]:
from functools import lru_cache

In [113]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [114]:
edit_distance('ABCDECG','ABCCEF')

3

In [115]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'A'): 'DEL G',
 ('ABCDECG', 'A

## Todo: Parse Solution is our homework

In [22]:
# @lru_cache(maxsize=2**10)
def edit_distance_2(string1, string2):
    
    if string1 == string2 : return 0, {}
    if len(string1) == 0: return len(string2), {(string1, string2):'ADD {}'.format(string2)}
    if len(string2) == 0: return len(string1), {(string1, string2):'DEL {}'.format(string1)}
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    rst_dict = {}
    
    del_dis, del_dict = edit_distance_2(string1[:-1], string2)
    del_op = 'DEL {}'.format(tail_s1)
    rst_dict[del_op] = del_dict
    
    add_dis, add_dict = edit_distance_2(string1, string2[:-1])
    add_op = 'ADD {}'.format(tail_s2)
    rst_dict[add_op] = add_dict
    
    candidates = [
        (del_dis + 1, del_op),  
        # string 1 delete tail
        (add_dis + 1, add_op),  
        # string 1 add tail of string2
    ]
    
    both_dis, both_dict = edit_distance_2(string1[:-1], string2[:-1])
    
    if tail_s1 == tail_s2:
        both_forward = (both_dis + 0, '')
        rst_dict[''] = both_dict
    else:
        sub_op = 'SUB {} => {}'.format(tail_s1, tail_s2)
        both_forward = (both_dis + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))
        rst_dict[sub_op] = both_dict

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    result_dict = rst_dict[operation]
    result_dict[(string1, string2)] = operation 
    
    return min_distance, result_dict

In [23]:
dis, op = edit_distance_2('AB','AC')
print(dis)
op

1


{('AB', 'AC'): 'SUB B => C'}

### 使用lru_cache
不明白为什么使用了lru_cache后结果与预期不符，可能是lru_cache不支持这种代码结构？

In [148]:
dis, op = edit_distance_2('ABCDECG','ABCCEF')
print(dis)
op

3


{('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'ABC'): 'DEL G',
 ('ABCDECG', 'ABCC'): 'DEL G',
 ('ABCDECG', 'ABCCE'): 'DEL G',
 ('ABCDECG', 'ABCCEF'): 'DEL G'}

### 不使用lru_cache

In [150]:
dis, op = edit_distance_2('ABCDECG','ABCCEF')
print(dis)
op

3


{('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'ABCCEF'): 'DEL G'}

# Problem Case 3: Pinyin Auto Correction Problem

In [206]:
chinese_dataset = 'article_9k.txt'

In [208]:
CHINESE_CHARATERS = open(chinese_dataset).read()

In [209]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [211]:
import pinyin

In [215]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [216]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [217]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [218]:
len(CHINESE_CHARATERS_COPYS)

129612256

In [219]:
import re

In [220]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [221]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [222]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [47]:
from collections import Counter, defaultdict

In [224]:
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [48]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [49]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [50]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [228]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [229]:
print(edits0('pinyin'))

{'pinyin'}


In [230]:
print(edits1('pinyin'))

{'yinyin', 'pinyio', 'pfnyin', 'pinycn', 'spinyin', 'pinyjn', 'pingyin', 'pinytin', 'pinjin', 'pinyxn', 'peinyin', 'pinuin', 'pinxin', 'pinyn', 'psnyin', 'pienyin', 'pdnyin', 'penyin', 'pinyyn', 'pinybn', 'pinxyin', 'pinygin', 'winyin', 'pineyin', 'pinyikn', 'ninyin', 'pinyzn', 'painyin', 'pinyinr', 'pinyxin', 'pinyinv', 'pinylin', 'phinyin', 'piqnyin', 'hinyin', 'pinyie', 'ainyin', 'cinyin', 'piniin', 'pinoin', 'pinyit', 'cpinyin', 'pinhyin', 'pignyin', 'pinsyin', 'pinyixn', 'pinlin', 'pintyin', 'pinyinu', 'zpinyin', 'lpinyin', 'pinoyin', 'ginyin', 'pcinyin', 'oinyin', 'pinysn', 'pixyin', 'mpinyin', 'pipnyin', 'pinyiin', 'pinrin', 'piunyin', 'pwinyin', 'pwnyin', 'pivnyin', 'pinyian', 'pijyin', 'pindin', 'pinyins', 'dinyin', 'pinyni', 'pinyivn', 'pinykn', 'tinyin', 'pinyij', 'pninyin', 'pinyipn', 'pinyis', 'vinyin', 'piyin', 'pibyin', 'pinyip', 'pinyiz', 'pfinyin', 'pibnyin', 'pinwyin', 'pinynn', 'pinyinf', 'pinlyin', 'panyin', 'pinyinx', 'pinein', 'pinyiun', 'pinyidn', 'piwnyin', 'sin

# Test

In [231]:
correct('yin')

'yin'

In [232]:
correct('yign')

'ying'

In [137]:
correct('shngqi')

'shngqi'

In [234]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [237]:
correct_sequence_pinyin('zhe sih yi ge ce shi')

'zhe shi yi ge ce shi'

In [238]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

In [1]:
import pinyin
import re
from collections import Counter

In [2]:
chinese_dataset = 'article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset).read()

# 获得所有拼音tokens
def chinese_to_pinyin_tokens(text):
    # 过滤得到汉字
    text = ''.join(re.findall('[\u4e00-\u9fa5]+',text))
    return pinyin.get(text, format='strip', delimiter=' ').split(' ')

In [3]:
pinyin_tokens = chinese_to_pinyin_tokens(CHINESE_CHARATERS)
pinyin_tokens_2 = [' '.join(pinyin_tokens[i:i+2]) for i in range(len(pinyin_tokens[:-2]))]
PINYIN_COUNT = Counter(pinyin_tokens)
PINYIN_COUNT_2 = Counter(pinyin_tokens_2)

In [6]:
PINYIN_COUNT.most_common(20)

[('shi', 860634),
 ('de', 809887),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441),
 ('zai', 334106),
 ('wei', 326301),
 ('hua', 304941),
 ('yu', 302949),
 ('she', 293312),
 ('he', 285689),
 ('bu', 281533),
 ('ri', 278379),
 ('jin', 278141),
 ('you', 277726),
 ('xian', 269047)]

In [9]:
len(PINYIN_COUNT)

411

In [199]:
@lru_cache(maxsize=2**10)
def split_sequence_pinyin(text):
    text = text.replace(' ', '')
    
    if text in PINYIN_COUNT: return [text]
    if text == '': return []
    candidates = []
    correct_candidates = []
    for i in range(1, 7, 1):
        if i > len(text):
            break
        word = text[:i]
        if word in PINYIN_COUNT:
            candidates.append([word] + split_sequence_pinyin(text[i:]))
        else:
            correct_candidates.append([correct(word)] + split_sequence_pinyin(text[i:]))
    
#     max(candidates,key=PINYIN_COUNT.get)
#     return max(candidates,key=lambda x: PINYIN_COUNT.get(x[0]) if len(x) == 1 else PINYIN_COUNT_2.get(x[0] + ' ' + x[1]))
    
#     print(text)
#     print(candidates)
#     print(correct_candidates)
    
    if len(candidates) > 0:
        result = get_best_candidate(candidates)
    elif len(correct_candidates) > 0:
        result = get_best_candidate(correct_candidates)
    else:
        result = []
        
#     print(result)
    return result

def get_best_candidate(candidates):
    result = []
    if len(candidates) == 0: return []
    for candi in candidates:
        if len(candi) == 1:
            result.append(PINYIN_COUNT.get(candi[0], 1) / len(pinyin_tokens))
        else:
            result.append(PINYIN_COUNT_2.get(candi[0] + ' ' + candi[1], 1) / len(pinyin_tokens_2))
#             rst = 1
#             for i in range(len(candi)-1):
#                 rst *= PINYIN_COUNT_2.get(candi[i] + ' ' + candi[i+1], 1) / len(pinyin_tokens_2)
#             # 概率需要开根号，否则拆分越短的词就概率越大
#             rst = rst**(1 / (len(candi)-1))
#             result.append(rst)
#     print(result)

    return candidates[result.index(max(result))]



In [200]:
split_sequence_pinyin('wosyaoshangqinghua')

['wo', 'yao', 'shang', 'qing', 'hua']

In [201]:
split_sequence_pinyin('wyaoshangqinghua')

['yao', 'shang', 'qing', 'hua']

In [202]:
split_sequence_pinyin.cache_clear()
split_sequence_pinyin('woyaoshngqinghua')

['wo', 'yao', 'shi', 'qing', 'hua']

### 这个题，精度如果要提高，感觉还是很难啊。还有好多异常错误无法纠正。。。